<a href="https://colab.research.google.com/github/hansjang/Py_Radar/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0Part4_P264.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 709, done.
remote: Counting objects: 100% (346/346), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 709 (delta 156), reused 237 (delta 115), pack-reused 363 (from 1)
Receiving objects: 100% (709/709), 11.46 MiB | 9.07 MiB/s, done.
Resolving deltas: 100% (341/341), done.


**1.단일표본 T-검정** : Trees 의 Height <BR>
- H0 : 평균 75와 같다
- H1 : 평균 75와 다르다

In [ ]:
import pandas as pd
import numpy as np

X_test = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/trees.csv')
print(X_test.info())
print(X_test.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Girth   31 non-null     float64
 1   Height  31 non-null     int64  
 2   Volume  31 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 872.0 bytes
None
           Girth     Height     Volume
count  31.000000  31.000000  31.000000
mean   13.248387  76.000000  30.170968
std     3.138139   6.371813  16.437846
min     8.300000  63.000000  10.200000
25%    11.050000  72.000000  19.400000
50%    12.900000  76.000000  24.200000
75%    15.250000  80.000000  37.300000
max    20.600000  87.000000  77.000000


In [ ]:
# 정규성 검정(shapiro-wilk)
# 데이터가 정규 분포를 따르는 지 확인하기 위해서 수행한다.
# 많은 통계 기법(t-검정, ANOVA, 회귀 분석 등)들이 데이터가 정규 분포를 따른다는 가정을 전제로 하기 때문이다.

from scipy.stats import shapiro

stat, p_value = shapiro(X_test['Height'])
print('w_stat : ', stat, 'p_value : ', p_value)

# Height(높이)의 p-value 가 유의수준 0.05 보다 크므로 정규성을 따를 가능성이 높다.

w_stat :  0.9654542507200756 p_value :  0.4034181746323222


In [ ]:
import scipy.stats as stats
from math import sqrt

# Height
H_mean = X_test['Height'].mean()
print(H_mean)

76.0


In [ ]:
t_score, p_value = stats.ttest_1samp(X_test['Height'], 75)
print('t_score : ', t_score, 'p_value : ', p_value)

# p-value 가 유의수준 0.05 보다 크므로 귀무가설(영가설)을 채택한다.
# 즉 평균 75와 같다고 볼 수 있다.

t_score :  0.8738116490580545 p_value :  0.38916223357338076


**2.독립표본 T-검정** : Toothgrowth

In [ ]:
data = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/toothgrowth.csv')

print(data.info())
print(data.describe())
print(data['supp'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   len     60 non-null     float64
 1   supp    60 non-null     object 
 2   dose    60 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.5+ KB
None
             len       dose
count  60.000000  60.000000
mean   18.813333   1.166667
std     7.649315   0.628872
min     4.200000   0.500000
25%    13.075000   0.500000
50%    19.250000   1.000000
75%    25.275000   2.000000
max    33.900000   2.000000
['VC' 'OJ']


In [ ]:
import scipy.stats as stats
from math import sqrt

# 독립표본 T-검정은 등분산성을 만족해야 하므로,
# Levene 검정을 사용한 등분산성 검정을 선행해야 한다.

stats.levene(data.loc[data.supp == 'VC', 'len'], data.loc[data.supp == 'OJ', 'len'])

# p-value가 유의수준 0.05보다 크므로 귀무가설을 채택한다.
# 즉 VC와 OJ 그룹간 len변수 분포는 등분산일 가능성이 높다.

LeveneResult(statistic=1.2135720656945064, pvalue=0.2751764616144053)

In [ ]:
data_VC = data.loc[data.supp == 'VC']
data_OJ = data.loc[data.supp == 'OJ']

VC_X = data_VC['len'].mean()
OJ_X = data_OJ['len'].mean()

print(round(VC_X, 2), round(OJ_X, 2))

16.96 20.66


In [ ]:
stats.ttest_ind(data_VC['len'], data_OJ['len'], equal_var=True)

TtestResult(statistic=-1.91526826869527, pvalue=0.06039337122412849, df=58.0)

In [ ]:
t_score, p_value = stats.ttest_ind(data_VC['len'], data_OJ['len'], equal_var=True)
print('t_score : ', round(t_score, 2), 'p_value : ', round(p_value, 2))

# p-value 가 유의수준 0.05 보다 크므로 귀무가설(영가설)을 채택한다.
# 오렌지 주스를 투여한 기니피그와 아스코르브산을 투여한 기니피그의 치아 길이 평균은 같다고 볼 수 있다.

t_score :  -1.92 p_value :  0.06


**3. 쌍체표본 T-검정** : Insectsprays <br>
- before_spr : 살충제 뿌리기 전 곤충의 수
- after_spr : 살충제 뿌린 뒤 곤충의 수

In [ ]:
data = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/insectsprays.csv')

print(data.info())
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   before_spr  36 non-null     int64
 1   after_spr   36 non-null     int64
dtypes: int64(2)
memory usage: 704.0 bytes
None
       before_spr  after_spr
count   36.000000  36.000000
mean    15.500000   3.500000
std      5.068108   2.348252
min      7.000000   0.000000
25%     11.750000   2.000000
50%     14.500000   3.000000
75%     19.250000   5.000000
max     26.000000  12.000000


In [ ]:
# [after_spr] - [before_spr] 의 표본 평균
diff = data['after_spr'] - data['before_spr']
diff_mean = diff.mean()
print(round(diff_mean, 2))

-12.0


In [ ]:
t_score, p_value = stats.ttest_rel(data['before_spr'], data['after_spr'], alternative='greater')
print('t_score : ', round(t_score, 2), 'p_value : ', round(p_value, 2))

# p-value가 유의수준 0.05보다 작으므로 귀무가설을 기각하고 대립가설을 채택한다.
# 즉 살충제의 효과가 있다

t_score :  14.89 p_value :  0.0


**4.일원분산분석(One-way ANOVA)** : iris

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats

data = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/iris.csv')

print(data.info())
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB
None
       sepal_length  sepal_width  petal_length  petal_width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.057333      3.758000     1.199333
std        0.828066     0.435866      1.765298     0.762238
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.50000

In [8]:
print(data['species'].unique())

['setosa' 'versicolor' 'virginica']


In [7]:
result = data.loc[data['species'] == 'versicolor', 'sepal_width'].mean()
print(round(result, 2))

2.77


In [10]:
X1 = data.loc[data['species'] == 'setosa', 'sepal_width']
X2 = data.loc[data['species'] == 'versicolor', 'sepal_width']
X3 = data.loc[data['species'] == 'virginica', 'sepal_width']

print(round(X1.mean(), 2), round(X2.mean(), 2), round(X3.mean(),2))

3.43 2.77 2.97


In [15]:
test_result = stats.f_oneway(X1, X2, X3)
print(round(test_result.statistic,2))
print(round(test_result.pvalue,2))

49.16
0.0


In [16]:
p_value = test_result.pvalue
print('p_value : ', round(p_value, 2))

if p_value >= 0.05:
  print('귀무가설 채택')
else:
  print('귀무가설 기각')

p_value :  0.0
귀무가설 기각


**5.카이제곱검정(적합도 검정)** : cellphone

In [18]:
import pandas as pd
import scipy.stats as stats

data = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/cellphone.csv')

print(data.info())
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      200 non-null    int64 
 1   제조사     200 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.2+ KB
None
               id
count  200.000000
mean   100.500000
std     57.879185
min      1.000000
25%     50.750000
50%    100.500000
75%    150.250000
max    200.000000


In [19]:
count_df = data.groupby('제조사').count()
print(count_df)

      id
제조사     
기타     5
삼성   117
애플    78


In [20]:
result = count_df.loc['삼성'] - count_df.loc['애플']
print(result)

id    39
dtype: int64


In [23]:
total_sum = len(data)
expected = [int(total_sum *0.1),
            int(total_sum*0.6),
            int(total_sum*0.3)]
print(expected)

[20, 120, 60]


In [25]:
observed = count_df['id'].values
test_result = stats.chisquare(observed, f_exp = expected)

result = round(test_result.statistic, 2)

print(result)

16.73


In [26]:
p_value = test_result.pvalue

print(round(p_value, 4))

if p_value >= 0.05:
  print('귀무가설 채택')
else :
  print('귀무가설 기각')

0.0002
귀무가설 기각


**6.카이제곱검정(독립성 검정)** : student

In [28]:
import pandas as pd
import scipy.stats as stats

data = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/student.csv')

data.head()

,ID,단과대학,수강 과목 수
0,1,경영대학,4개 이하
1,2,경영대학,4개 이하
2,3,경영대학,4개 이하
3,4,경영대학,4개 이하
4,5,경영대학,4개 이하


In [29]:
table = pd.crosstab(data['단과대학'], data['수강 과목 수'])
print(table)

수강 과목 수  4개 이하  5개  6개 이상
단과대학                     
경영대학        12  78     30
공과대학        30  42     28
자연대학        30  42      8


In [30]:
result = table[table['6개 이상']==table['6개 이상'].max()].index[0]
print(result)

경영대학


In [31]:
X1 = table.loc['경영대학', :]
X2 = table.loc['공과대학', :]
X3 = table.loc['자연대학', :]

test_result = stats.chi2_contingency([X1, X2, X3])
result = test_result.statistic

print(round(result, 2))

30.13


In [32]:
p_value = test_result.pvalue

print(round(p_value, 4))

if p_value >= 0.05:
  print('귀무가설 채택')
else:
  print('귀무가설 기각')

0.0
귀무가설 기각
